In [1]:
import pandas as pd
import scipy.stats as stats

from sklearn import metrics, model_selection
from catboost import CatBoostClassifier

In [2]:
df_train = pd.read_csv('../../Feature_Encoding/data/train_loo_encoding.csv')
df_test = pd.read_csv('../../Feature_Encoding/data/test_loo_encoding.csv')
train = pd.read_csv('../../Feature_Engineering/data/other-cleaned_train.csv')

In [3]:
def cross_val(model, x_train, y_train):
    score_cross_val = model_selection.cross_val_score(model, x_train, y_train, cv=5)
    print(score_cross_val.mean())

In [4]:
#Realiza busqueda completa combinando los parametros
def catboost(x_train, y_train, x_validation, y_validation):
    catb_classifier = CatBoostClassifier()
    params_catb = {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.05, 0.1], 'l2_leaf_reg': [0, 1, 5]}    
    catb_gs = model_selection.GridSearchCV(catb_classifier, params_catb, cv=5)
    catb_gs.fit(x_train, y_train)
    catb_best = catb_gs.best_estimator_
    print(catb_gs.best_params_)
    print('catb: {}'.format(catb_best.score(x_validation, y_validation)))
    return catb_best

In [5]:
#Realiza busqueda random dentro de los parametros validos
def catboost2(x_train, y_train, x_validation, y_validation):
    catb_classifier = CatBoostClassifier()
    params_catb_2={
        'n_estimators':stats.randint(10,500),'learning_rate':stats.uniform(0.01,0.3),
        'subsample':stats.uniform(0.3,0.7),'l2_leaf_reg':[1,5,10],
        'max_depth':[3,10,6],'colsample_bylevel':stats.uniform(0.,0.6)
    }
    
    catb_rs = model_selection.RandomizedSearchCV(CatBoostClassifier(),
                          param_distributions=params_catb_2,
                          cv=2,
                          scoring='neg_log_loss',
                          verbose=1,
                          n_iter=200)
    
    catb_rs.fit(x_train, y_train)
    catb_best = catb_rs.best_estimator_
    print(catb_rs.best_params_)
    print('catb: {}'.format(catb_best.score(x_validation, y_validation)))
    return catb_best

In [6]:
def test_model(model, x_test, y_test):
    predictions = model.predict_proba(x_test)[:,1]
    logloss = metrics.log_loss(y_test, predictions)
    accuracy = metrics.accuracy_score(y_test, predictions.round())
    print("Accuracy: %.2f%%, Logloss: %.2f" % (accuracy*100.0, logloss))

In [7]:
y = train.Target
x_train, x_validation, y_train, y_validation = model_selection.train_test_split(df_train, y, test_size=0.3, stratify=y)

In [ ]:
catboost_2 = catboost2(x_train, y_train, x_validation, y_validation)
test_model(catboost_2, x_validation, y_validation)
cross_val(catboost_2, x_train, y_train)

Fitting 2 folds for each of 200 candidates, totalling 400 fits
0:	learn: 0.6438507	total: 102ms	remaining: 20.1s
1:	learn: 0.6010493	total: 113ms	remaining: 11s
2:	learn: 0.5577589	total: 118ms	remaining: 7.64s
3:	learn: 0.5265126	total: 122ms	remaining: 5.89s
4:	learn: 0.5077614	total: 127ms	remaining: 4.89s
5:	learn: 0.4913589	total: 131ms	remaining: 4.17s
6:	learn: 0.4759667	total: 134ms	remaining: 3.64s
7:	learn: 0.4551634	total: 137ms	remaining: 3.24s
8:	learn: 0.4439595	total: 140ms	remaining: 2.92s
9:	learn: 0.4237643	total: 142ms	remaining: 2.66s
10:	learn: 0.4099757	total: 145ms	remaining: 2.44s
11:	learn: 0.4002584	total: 147ms	remaining: 2.26s
12:	learn: 0.3901054	total: 150ms	remaining: 2.12s
13:	learn: 0.3743920	total: 152ms	remaining: 1.99s
14:	learn: 0.3686542	total: 154ms	remaining: 1.87s
15:	learn: 0.3608307	total: 157ms	remaining: 1.78s
16:	learn: 0.3572428	total: 160ms	remaining: 1.69s
17:	learn: 0.3526092	total: 162ms	remaining: 1.61s
18:	learn: 0.3470188	total: 164

0:	learn: 0.6445649	total: 5.01ms	remaining: 983ms
1:	learn: 0.6063122	total: 8.64ms	remaining: 843ms
2:	learn: 0.5791249	total: 11.8ms	remaining: 760ms
3:	learn: 0.5357071	total: 14.4ms	remaining: 693ms
4:	learn: 0.5127214	total: 16.7ms	remaining: 642ms
5:	learn: 0.4791160	total: 20.3ms	remaining: 646ms
6:	learn: 0.4621155	total: 23.4ms	remaining: 634ms
7:	learn: 0.4454929	total: 25.6ms	remaining: 605ms
8:	learn: 0.4330610	total: 27.8ms	remaining: 582ms
9:	learn: 0.4220048	total: 30.4ms	remaining: 569ms
10:	learn: 0.4123848	total: 33.1ms	remaining: 559ms
11:	learn: 0.3993246	total: 37.5ms	remaining: 578ms
12:	learn: 0.3909604	total: 40.7ms	remaining: 576ms
13:	learn: 0.3844155	total: 43.2ms	remaining: 565ms
14:	learn: 0.3777021	total: 45.8ms	remaining: 556ms
15:	learn: 0.3666477	total: 48.1ms	remaining: 544ms
16:	learn: 0.3619694	total: 53.6ms	remaining: 568ms
17:	learn: 0.3582495	total: 56.3ms	remaining: 560ms
18:	learn: 0.3521042	total: 58.7ms	remaining: 550ms
19:	learn: 0.3490454	t

0:	learn: 0.5554892	total: 14.1ms	remaining: 2.82s
1:	learn: 0.4843563	total: 26.5ms	remaining: 2.63s
2:	learn: 0.4258999	total: 40.4ms	remaining: 2.67s
3:	learn: 0.3905351	total: 52.5ms	remaining: 2.58s
4:	learn: 0.3684299	total: 66.9ms	remaining: 2.62s
5:	learn: 0.3458555	total: 78.1ms	remaining: 2.54s
6:	learn: 0.3335789	total: 88.9ms	remaining: 2.46s
7:	learn: 0.3220982	total: 99.5ms	remaining: 2.4s
8:	learn: 0.3144224	total: 116ms	remaining: 2.47s
9:	learn: 0.3037585	total: 125ms	remaining: 2.38s
10:	learn: 0.2960741	total: 132ms	remaining: 2.28s
11:	learn: 0.2889125	total: 140ms	remaining: 2.21s
12:	learn: 0.2847907	total: 154ms	remaining: 2.22s
13:	learn: 0.2775171	total: 176ms	remaining: 2.35s
14:	learn: 0.2741895	total: 185ms	remaining: 2.29s
15:	learn: 0.2687146	total: 190ms	remaining: 2.2s
16:	learn: 0.2636633	total: 194ms	remaining: 2.1s
17:	learn: 0.2594017	total: 198ms	remaining: 2.01s
18:	learn: 0.2578471	total: 202ms	remaining: 1.93s
19:	learn: 0.2539619	total: 205ms	re

168:	learn: 0.0793437	total: 800ms	remaining: 152ms
169:	learn: 0.0785945	total: 804ms	remaining: 147ms
170:	learn: 0.0777894	total: 807ms	remaining: 142ms
171:	learn: 0.0775468	total: 811ms	remaining: 137ms
172:	learn: 0.0771359	total: 814ms	remaining: 132ms
173:	learn: 0.0766406	total: 818ms	remaining: 127ms
174:	learn: 0.0763746	total: 822ms	remaining: 122ms
175:	learn: 0.0758983	total: 826ms	remaining: 117ms
176:	learn: 0.0754633	total: 829ms	remaining: 112ms
177:	learn: 0.0752075	total: 833ms	remaining: 108ms
178:	learn: 0.0748370	total: 837ms	remaining: 103ms
179:	learn: 0.0744695	total: 841ms	remaining: 98.1ms
180:	learn: 0.0740452	total: 844ms	remaining: 93.3ms
181:	learn: 0.0735574	total: 848ms	remaining: 88.5ms
182:	learn: 0.0728892	total: 851ms	remaining: 83.7ms
183:	learn: 0.0724434	total: 855ms	remaining: 79ms
184:	learn: 0.0720229	total: 858ms	remaining: 74.2ms
185:	learn: 0.0715650	total: 862ms	remaining: 69.5ms
186:	learn: 0.0711248	total: 865ms	remaining: 64.8ms
187:	l

173:	learn: 0.0671217	total: 802ms	remaining: 124ms
174:	learn: 0.0666432	total: 806ms	remaining: 120ms
175:	learn: 0.0664465	total: 810ms	remaining: 115ms
176:	learn: 0.0660357	total: 814ms	remaining: 110ms
177:	learn: 0.0656861	total: 817ms	remaining: 106ms
178:	learn: 0.0654419	total: 820ms	remaining: 101ms
179:	learn: 0.0651026	total: 824ms	remaining: 96.1ms
180:	learn: 0.0647964	total: 827ms	remaining: 91.4ms
181:	learn: 0.0645118	total: 832ms	remaining: 86.8ms
182:	learn: 0.0641057	total: 835ms	remaining: 82.1ms
183:	learn: 0.0637184	total: 839ms	remaining: 77.5ms
184:	learn: 0.0631094	total: 842ms	remaining: 72.8ms
185:	learn: 0.0628054	total: 847ms	remaining: 68.3ms
186:	learn: 0.0622338	total: 850ms	remaining: 63.6ms
187:	learn: 0.0618185	total: 853ms	remaining: 59ms
188:	learn: 0.0614744	total: 857ms	remaining: 54.4ms
189:	learn: 0.0613171	total: 861ms	remaining: 49.8ms
190:	learn: 0.0609797	total: 864ms	remaining: 45.2ms
191:	learn: 0.0607704	total: 868ms	remaining: 40.7ms
1

...

300:	learn: 0.1788350	total: 1.63s	remaining: 950ms
301:	learn: 0.1785311	total: 1.63s	remaining: 945ms
302:	learn: 0.1781530	total: 1.63s	remaining: 938ms
303:	learn: 0.1780197	total: 1.64s	remaining: 932ms
304:	learn: 0.1778350	total: 1.64s	remaining: 925ms
305:	learn: 0.1775693	total: 1.64s	remaining: 919ms
306:	learn: 0.1773565	total: 1.65s	remaining: 913ms
307:	learn: 0.1770888	total: 1.65s	remaining: 907ms
308:	learn: 0.1768659	total: 1.66s	remaining: 900ms
309:	learn: 0.1766408	total: 1.66s	remaining: 894ms
310:	learn: 0.1764894	total: 1.66s	remaining: 888ms
311:	learn: 0.1761525	total: 1.67s	remaining: 882ms
312:	learn: 0.1760981	total: 1.67s	remaining: 876ms
313:	learn: 0.1759860	total: 1.68s	remaining: 870ms
314:	learn: 0.1757119	total: 1.68s	remaining: 863ms
315:	learn: 0.1754656	total: 1.68s	remaining: 857ms
316:	learn: 0.1752701	total: 1.69s	remaining: 851ms
317:	learn: 0.1752231	total: 1.69s	remaining: 845ms
318:	learn: 0.1751099	total: 1.69s	remaining: 839ms
319:	learn: 

0:	learn: 0.6517327	total: 5.74ms	remaining: 1.72s
1:	learn: 0.6198892	total: 9.87ms	remaining: 1.48s
2:	learn: 0.5938729	total: 15.6ms	remaining: 1.55s
3:	learn: 0.5687616	total: 21.8ms	remaining: 1.62s
4:	learn: 0.5463219	total: 26.3ms	remaining: 1.55s
5:	learn: 0.5273365	total: 34.1ms	remaining: 1.68s
6:	learn: 0.5128035	total: 39.6ms	remaining: 1.66s
7:	learn: 0.4989250	total: 43.8ms	remaining: 1.6s
8:	learn: 0.4858586	total: 52.6ms	remaining: 1.71s
9:	learn: 0.4739357	total: 58.5ms	remaining: 1.7s
10:	learn: 0.4612703	total: 64.2ms	remaining: 1.69s
11:	learn: 0.4491225	total: 70.2ms	remaining: 1.69s
12:	learn: 0.4408767	total: 76.9ms	remaining: 1.7s
13:	learn: 0.4254256	total: 89.8ms	remaining: 1.84s
14:	learn: 0.4177677	total: 102ms	remaining: 1.94s
15:	learn: 0.4062249	total: 105ms	remaining: 1.86s
16:	learn: 0.3989235	total: 116ms	remaining: 1.94s
17:	learn: 0.3917367	total: 123ms	remaining: 1.94s
18:	learn: 0.3863599	total: 128ms	remaining: 1.9s
19:	learn: 0.3824445	total: 135

175:	learn: 0.2504584	total: 1.2s	remaining: 853ms
176:	learn: 0.2502573	total: 1.21s	remaining: 845ms
177:	learn: 0.2499746	total: 1.21s	remaining: 836ms
178:	learn: 0.2497135	total: 1.21s	remaining: 827ms
179:	learn: 0.2494469	total: 1.22s	remaining: 819ms
180:	learn: 0.2490658	total: 1.22s	remaining: 810ms
181:	learn: 0.2487856	total: 1.23s	remaining: 802ms
182:	learn: 0.2484857	total: 1.23s	remaining: 794ms
183:	learn: 0.2482542	total: 1.24s	remaining: 786ms
184:	learn: 0.2479424	total: 1.24s	remaining: 777ms
185:	learn: 0.2476912	total: 1.24s	remaining: 769ms
186:	learn: 0.2473788	total: 1.25s	remaining: 761ms
187:	learn: 0.2471171	total: 1.26s	remaining: 761ms
188:	learn: 0.2468494	total: 1.27s	remaining: 756ms
189:	learn: 0.2465545	total: 1.28s	remaining: 751ms
190:	learn: 0.2464195	total: 1.3s	remaining: 747ms
191:	learn: 0.2461016	total: 1.3s	remaining: 738ms
192:	learn: 0.2458002	total: 1.3s	remaining: 730ms
193:	learn: 0.2456694	total: 1.31s	remaining: 726ms
194:	learn: 0.24

61:	learn: 0.2914456	total: 195ms	remaining: 751ms
62:	learn: 0.2904434	total: 199ms	remaining: 752ms
63:	learn: 0.2898477	total: 203ms	remaining: 752ms
64:	learn: 0.2894514	total: 208ms	remaining: 756ms
65:	learn: 0.2887134	total: 217ms	remaining: 772ms
66:	learn: 0.2881561	total: 220ms	remaining: 770ms
67:	learn: 0.2869469	total: 224ms	remaining: 768ms
68:	learn: 0.2863002	total: 228ms	remaining: 766ms
69:	learn: 0.2854737	total: 231ms	remaining: 764ms
70:	learn: 0.2846828	total: 235ms	remaining: 762ms
71:	learn: 0.2844295	total: 239ms	remaining: 760ms
72:	learn: 0.2832588	total: 243ms	remaining: 758ms
73:	learn: 0.2827664	total: 246ms	remaining: 755ms
74:	learn: 0.2822409	total: 250ms	remaining: 753ms
75:	learn: 0.2813091	total: 253ms	remaining: 750ms
76:	learn: 0.2806623	total: 259ms	remaining: 752ms
77:	learn: 0.2800127	total: 262ms	remaining: 748ms
78:	learn: 0.2791844	total: 266ms	remaining: 746ms
79:	learn: 0.2787856	total: 270ms	remaining: 745ms
80:	learn: 0.2784027	total: 273

247:	learn: 0.2156703	total: 979ms	remaining: 209ms
248:	learn: 0.2155325	total: 984ms	remaining: 206ms
249:	learn: 0.2152534	total: 988ms	remaining: 201ms
250:	learn: 0.2151495	total: 992ms	remaining: 198ms
251:	learn: 0.2148625	total: 999ms	remaining: 194ms
252:	learn: 0.2146035	total: 1.01s	remaining: 191ms
253:	learn: 0.2143676	total: 1.01s	remaining: 187ms
254:	learn: 0.2142436	total: 1.02s	remaining: 184ms
255:	learn: 0.2140981	total: 1.03s	remaining: 181ms
256:	learn: 0.2139450	total: 1.04s	remaining: 177ms
257:	learn: 0.2137178	total: 1.04s	remaining: 174ms
258:	learn: 0.2135730	total: 1.06s	remaining: 171ms
259:	learn: 0.2134259	total: 1.06s	remaining: 167ms
260:	learn: 0.2130992	total: 1.06s	remaining: 163ms
261:	learn: 0.2128040	total: 1.07s	remaining: 159ms
262:	learn: 0.2125411	total: 1.07s	remaining: 155ms
263:	learn: 0.2122506	total: 1.07s	remaining: 151ms
264:	learn: 0.2119906	total: 1.08s	remaining: 147ms
265:	learn: 0.2115376	total: 1.08s	remaining: 143ms
266:	learn: 

In [ ]:
y_pred = catboost_2.predict_proba(df_test)[:,1]
submission_catboost = pd.DataFrame(data={'Opportunity_ID':df_test['Opportunity_ID'], 'Target': y_pred})
submission_catboost = submission_catboost.groupby("Opportunity_ID").agg({"Target":"mean"}).reset_index()
submission_catboost.to_csv('../submits/loo_catboost.csv', index=False)